In [ ]:
# install the relevant packages
import requests
import pandas as pd
from mplsoccer import Pitch
import matplotlib.pyplot as plt
import json
import os
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
# Scrape all of the shot data from a given fixture from SofaScore
url = 'https://www.sofascore.com/football/match/arsenal-chelsea/NR#id:12436890'

# the id for the match is listed at the end of the URL
match_id = url.split('id:')[-1]

match_id

In [ ]:
# We can extract the team names from the URL

url_team_names = url.split('/')[5]

url_team_names

In [ ]:
# We can define our Home & Away Team names

home_team_name = url_team_names.split('-')[0].capitalize()
away_team_name = url_team_names.split('-')[-1].capitalize()

print(f'{home_team_name} vs {away_team_name}')

In [ ]:
# execute a curl command to get the shot data - will need to sub in the match_id
response = json.loads(os.popen(
  f'curl -H "Host: api.sofascore.com" -H "Accept: */*" -H "User-Agent: curl/8.1.2" https://api.sofascore.com/api/v1/event/{match_id}/shotmap').read())['shotmap']

In [ ]:
# turn this data into a df

df = pd.DataFrame(response)

df.columns

df.head()

In [ ]:
# Extract the x & y values from the shot coordinates & create indipendant columns for each
df['x'] = df['playerCoordinates'].apply(lambda coord: coord['x'])
df['y'] = df['playerCoordinates'].apply(lambda coord: coord['y'])

# Extract player names from the nested dictionary
df['player_name'] = df['player'].apply(lambda x: x['name'] if isinstance(x, dict) and 'name' in x else None)

df.head()

In [ ]:
# We'll need to adjust these coordinates from the sofascore pitch size to the statsbomb pitch size to plot them acurately on our pitch

# Sofascore uses pitch size 100X100
# Statsbomb uses pitch size 120X80

sofascore_max_x = 100

sofascore_max_y = 100

# Scale the x-values from 0-100 (sofascore map) to 0-120 (sb map)

df['x_scaled'] = (df['x'] / sofascore_max_x) * 120

# Scale the y-values from 0-100 (sofascore map) to 0-80 (sb map)

df['y_scaled'] = (df['y'] / sofascore_max_y) * 80

In [ ]:
# Separate home and away data for seperate plotting
home_data = df[df['isHome']]
away_data = df[~df['isHome']]

In [ ]:
# Invert the x coordinates for away team data to show plots on the right side of the pitch
away_data['x_scaled'] = 120 - away_data['x_scaled']

In [ ]:
from mplsoccer import Pitch
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Initialize the pitch (using a standard StatsBomb layout)
pitch = Pitch(pitch_type='statsbomb', pitch_color='white', line_color='black', line_zorder=2)

# Create figure and axis
fig, ax = plt.subplots(figsize=(10, 6))  # One figure, one axis

# Combine home and away team shots for the heatmap data
all_shots = pd.concat([home_data, away_data])

# Define the custom colormap (white to light purple)
cmap = LinearSegmentedColormap.from_list('custom_cmap', ['white', 'purple'])

# Plot heatmap (with lower zorder to keep it behind the pitch lines)
kde = pitch.kdeplot(
    all_shots['x_scaled'],  # x-coordinates of all shots, away team data is still inverted on the pitch
    all_shots['y_scaled'],  # y-coordinates of all shots
    ax=ax,
    fill=True, # cover the whole pitch
    cmap=cmap,  # Custom colormap
    n_levels=100,  # Smooth heatmap
    thresh=0,  # Include low-density areas
    zorder=1  # Draw heatmap beneath pitch lines
)

# Redraw the pitch lines (with higher zorder so they are above the heatmap)
pitch.draw(ax=ax)

# Plot home team shots
for index, shot in home_data.iterrows():
    x_coord = shot['x_scaled']
    y_coord = shot['y_scaled']
    color = 'green' if shot['shotType'] == 'goal' else 'white'
    zorder = 3 if shot['shotType'] == 'goal' else 2  # Keep above heatmap
    ax.scatter(x=x_coord, y=y_coord, s=1000 * shot['xg'], ec='black', c=color, zorder=zorder)

    # Add a label to the goals for the player who scored
    if shot['shotType'] == 'goal':  # Only label goals
        ax.text(
            x_coord,  # x-coordinate of the text
            y_coord - 3,  # y-coordinate of the text
            shot['player_name'],  # Text to display
            ha='center', 
            va='center', 
            fontfamily='monospace', 
            fontsize=8,
            zorder=10,
            bbox=dict(facecolor='lightgray', edgecolor='black', alpha=0.8)  # Add background with opacity
        )

# Plot away team shots
for index, shot in away_data.iterrows():
    x_coord = shot['x_scaled']
    y_coord = shot['y_scaled']
    color = 'green' if shot['shotType'] == 'goal' else 'white'
    zorder = 3 if shot['shotType'] == 'goal' else 2  # Keep above heatmap
    ax.scatter(x=x_coord, y=y_coord, s=1000 * shot['xg'], ec='black', c=color, zorder=zorder)

    # Add a label to the goals for the player who scored
    if shot['shotType'] == 'goal':  # Only label goals
        ax.text(
            x_coord,  # x-coordinate of the text
            y_coord - 3.5,  # y-coordinate of the text
            shot['player_name'],  # Text to display
            ha='center', 
            va='center', 
            fontfamily='monospace', 
            fontsize=8,
            zorder=10,
            bbox=dict(facecolor='lightgray', edgecolor='black', alpha=0.7)  # Add background with opacity
        )

# Add a Title
ax.set_title(f"{home_team_name} v {away_team_name}\nShot Map with Shot Heatmap", fontsize=20, y=1)

# Add the Team names
ax.text(30, 10, f'{home_team_name} Shots', ha='center', fontsize=16)
ax.text(90, 10, f'{away_team_name} Shots', ha='center', fontsize=16)

# Display
plt.tight_layout()
plt.show()


df.columns

In [ ]:
df.columns